# Preamble

In [1]:
# Essentials
import os, sys, glob
import pandas as pd
import numpy as np
import nibabel as nib

# Stats
import scipy as sp
from scipy import stats
import statsmodels.api as sm
import pingouin as pg

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['svg.fonttype'] = 'none'

In [2]:
sys.path.append('/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_T1/code/func/')
from proj_environment import set_proj_env
sys.path.append('/Users/lindenmp/Dropbox/Work/git/pyfunc/')
from func import get_synth_cov

In [3]:
train_test_str = 'squeakycleanExclude'
exclude_str = 't1Exclude' # 't1Exclude' 'fsFinalExclude'
parc_str = 'schaefer' # 'schaefer' 'lausanne'
parc_scale = 400 # 200 400 | 60 125 250
extra_str = ''
parcel_names, parcel_loc, drop_parcels, num_parcels, yeo_idx, yeo_labels = set_proj_env(train_test_str = train_test_str, exclude_str = exclude_str,
                                                                            parc_str = parc_str, parc_scale = parc_scale, extra_str = extra_str)

In [4]:
print(os.environ['MODELDIR_BASE'])
print(os.environ['MODELDIR'])

/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_T1/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_400
/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_T1/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_400


## Load data

In [5]:
# Load data
df = pd.read_csv(os.path.join(os.environ['MODELDIR_BASE'], 'df_pheno.csv'))
df.set_index(['bblid', 'scanid'], inplace = True)

# df_node = pd.read_csv(os.path.join(os.environ['MODELDIR'], 'df_node_clean.csv'))
df_node = pd.read_csv(os.path.join(os.environ['MODELDIR'], 'df_node_base.csv'))
df_node.set_index(['bblid', 'scanid'], inplace = True)

# adjust sex to 0 and 1
df['sex_adj'] = df.sex - 1
print(df.shape)
print(df_node.shape)

(1393, 51)
(1393, 801)


/Users/lindenmp/opt/anaconda3/envs/NormativeNeuroDev_CrossSec_T1/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.3, the latest is 0.3.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [6]:
print('Train:', np.sum(df[train_test_str] == 0), 'Test:', np.sum(df[train_test_str] == 1))

Train: 410 Test: 983


In [7]:
df.head()

,,squeakycleanExclude,ageAtScan1,ageAtScan1_Years,sex,race2,handednessv2,dti64MeanAbsRMS,dti64MeanRelRMS,dti64MaxAbsRMS,dti64MaxRelRMS,...,goassessSmryBeh,goassessSmryAdd,goassessSmryOdd,goassessSmryCon,goassessSmryPrimePos1,goassessSmryPrimeTot,goassessSmryPrimePos2,goassessSmryPsychOverallRtg,goassessDxpmr7,sex_adj
bblid,scanid,,,,,,,,,,,,,,,,,,,,,
80961,2632,1,259,21.6,1,1,2,0.697923,0.631253,1.073880,1.154490,...,4,1,4,1,0,37,1,4,PS,0
80199,2637,0,244,20.3,1,3,1,0.308559,0.149356,0.600316,0.334249,...,1,1,1,0,0,18,0,2,PS,0
80179,2643,1,254,21.2,2,1,1,1.030570,0.398904,1.770740,0.634785,...,1,1,0,0,0,0,0,2,TD,1
80812,2646,1,247,20.6,2,2,1,1.404950,1.404310,2.139150,3.436820,...,1,0,1,1,0,31,1,4,PS,1
80607,2647,1,252,21.0,1,2,1,0.376161,0.356844,0.645857,0.740534,...,4,0,4,0,0,4,0,4,OP,0


In [8]:
df_node.head()

,,squeakycleanExclude,ct_0,ct_1,ct_2,ct_3,ct_4,ct_5,ct_6,ct_7,ct_8,...,vol_390,vol_391,vol_392,vol_393,vol_394,vol_395,vol_396,vol_397,vol_398,vol_399
bblid,scanid,,,,,,,,,,,,,,,,,,,,,
80961,2632,1,4.059796,5.224811,3.974109,4.175344,3.763810,3.859499,4.831972,4.220583,3.602131,...,1658.0,3047.0,714.0,2388.0,1153.0,1038.0,1249.0,1346.0,1296.0,774.0
80199,2637,0,3.984655,3.580555,3.680155,3.390238,3.768151,4.350763,3.254272,3.868829,3.328264,...,2078.0,2837.0,1024.0,2076.0,1125.0,1288.0,829.0,1506.0,699.0,854.0
80179,2643,1,3.456456,3.712380,3.886699,3.087788,2.230043,3.583135,2.900838,4.383651,3.490465,...,1273.0,2327.0,798.0,1149.0,1169.0,1629.0,835.0,690.0,1031.0,580.0
80812,2646,1,3.458441,3.805802,3.131799,3.102077,3.033641,3.971838,3.728316,3.902617,3.817542,...,1594.0,2676.0,589.0,2422.0,1542.0,1294.0,952.0,1274.0,662.0,761.0
80607,2647,1,3.374282,3.259752,4.093978,3.402414,2.782389,3.900980,3.710548,4.468545,2.674039,...,1632.0,2581.0,683.0,1757.0,1297.0,1054.0,792.0,1336.0,745.0,560.0


# Prepare files for normative modelling

In [9]:
# Note, 'ageAtScan1_Years' is assumed to be covs[0] and 'sex_adj' is assumed to be covs[1]
# if more than 2 covs are to be used, append to the end and age/sex will be duplicated accordingly in the forward model
covs = ['ageAtScan1_Years', 'sex_adj']

print(covs)
num_covs = len(covs)
print(num_covs)

['ageAtScan1_Years', 'sex_adj']
2


In [10]:
extra_str_2 = ''

## Primary model (train/test split)

In [11]:
# Create subdirectory for specific normative model -- labeled according to parcellation/resolution choices and covariates
normativedir = os.path.join(os.environ['MODELDIR'], '+'.join(covs) + extra_str_2 + '/')
print(normativedir)
if not os.path.exists(normativedir): os.mkdir(normativedir);

/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_T1/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_400/ageAtScan1_Years+sex_adj/


In [12]:
# Write out training
df[df[train_test_str] == 0].to_csv(os.path.join(normativedir, 'train.csv'))
df[df[train_test_str] == 0].to_csv(os.path.join(normativedir, 'cov_train.txt'), columns = covs, sep = ' ', index = False, header = False)

# Write out test
df[df[train_test_str] == 1].to_csv(os.path.join(normativedir, 'test.csv'))
df[df[train_test_str] == 1].to_csv(os.path.join(normativedir, 'cov_test.txt'), columns = covs, sep = ' ', index = False, header = False)

In [13]:
# Write out training
resp_train = df_node[df_node[train_test_str] == 0].drop(train_test_str, axis = 1)
mask = np.all(np.isnan(resp_train), axis = 1)
if np.any(mask): print("Warning: NaNs in response train")
resp_train.to_csv(os.path.join(normativedir, 'resp_train.csv'))
resp_train.to_csv(os.path.join(normativedir, 'resp_train.txt'), sep = ' ', index = False, header = False)

# Write out test
resp_test = df_node[df_node[train_test_str] == 1].drop(train_test_str, axis = 1)
mask = np.all(np.isnan(resp_test), axis = 1)
if np.any(mask): print("Warning: NaNs in response train")
resp_test.to_csv(os.path.join(normativedir, 'resp_test.csv'))
resp_test.to_csv(os.path.join(normativedir, 'resp_test.txt'), sep = ' ', index = False, header = False)

print(str(resp_train.shape[1]) + ' features written out for normative modeling')

800 features written out for normative modeling


### Forward variants

In [14]:
fwddir = os.path.join(normativedir, 'forward/')
if not os.path.exists(fwddir): os.mkdir(fwddir)

# Synthetic cov data
x = get_synth_cov(df, cov = 'ageAtScan1_Years', stp = 1)

if 'sex_adj' in covs:
    # Produce gender dummy variable for one repeat --> i.e., to account for two runs of ages, one per gender
    gender_synth = np.concatenate((np.ones(x.shape),np.zeros(x.shape)), axis = 0)

# concat
synth_cov = np.concatenate((np.matlib.repmat(x, 2, 1), np.matlib.repmat(gender_synth, 1, 1)), axis = 1)
print(synth_cov.shape)

# write out
np.savetxt(os.path.join(fwddir, 'synth_cov_test.txt'), synth_cov, delimiter = ' ', fmt = ['%.1f', '%.d'])

(32, 2)


### Cross-val variant

In [15]:
# Create subdirectory for specific normative model -- labeled according to parcellation/resolution choices and covariates
cvdir = os.path.join(normativedir, 'cv/')
if not os.path.exists(cvdir): os.mkdir(cvdir)